In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU
from tensorflow.keras.optimizers import Adam

In [2]:
# 1727, 191
train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/1727-nonsquare-train-from-1918-nonsquare-spmm-over-3s.csv')
test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/d-optimal-of-spmm/train-test-csv/191-nonsquare-test-from-1918-nonsquare-spmm-over-3s.csv')

In [3]:
# feature 1개 추가

# Train + Valid
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_train = train['bz_smsm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_test = test['bz_smsm']

In [4]:
# 데이터 표준화(Standardization)
from sklearn.preprocessing import StandardScaler

# 변형 객체 생성
std_scaler = StandardScaler()

# 훈련데이터의 모수 분포 저장
std_scaler.fit(X_train)

# 훈련 데이터 스케일링
X_train_scaled = std_scaler.transform(X_train)

# 테스트 데이터의 스케일링
X_test_scaled = std_scaler.transform(X_test)

In [5]:
from sklearn.metrics import mean_squared_error

def mape_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def rmse_error(y_true, y_pred):
    rmse = np.sqrt(np.mean(np.square(y_pred - y_true))) 
    return rmse

### Xgbregressor

In [6]:
import xgboost as xgb

# Train + Valid cross-validation을 거친, 최적의 하이퍼파라미터를 사용
xgbregressor_model = xgb.XGBRegressor(
objective = 'reg:squarederror',
max_depth=int(18),
learning_rate=0.04,
n_estimators=int(95),
subsample=0.5625626271955027,
reg_lambda = 0.7058132473615808,    
colsample_bytree=0.9571507504641366,
n_jobs=-1
                             )
# 모델 훈련
xgbregressor_model.fit(X_train, y_train)

XGBRegressor(colsample_bytree=0.9571507504641366, learning_rate=0.04,
             max_depth=18, n_estimators=95, n_jobs=-1,
             objective='reg:squarederror', reg_lambda=0.7058132473615808,
             subsample=0.5625626271955027)

### dnn

In [7]:
# 모델 생성
def build_model():

    model=Sequential()

    model.add(Dense(256, activation="relu", input_shape=(X_train.shape[1],)))  
    model.add(Dense(512, activation="relu"))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1))
    
    optimizer = Adam(lr=0.01)
    
    model.compile(optimizer=optimizer ,
                  loss='mape',
                  metrics=['mape'])
    return model

dnn_model = build_model()

In [8]:
# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

early_stop = keras.callbacks.EarlyStopping(monitor='val_mape', patience=150)

EPOCHS = 100000

dnn_model.fit(X_train_scaled, 
                y_train,
                epochs=EPOCHS, 
                validation_split = 0.1, 
                verbose =0, 
                callbacks=[early_stop, PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
...................

### rfr

In [9]:
# from sklearn.ensemble import RandomForestRegressor

# rfr_model = RandomForestRegressor(
# criterion='mse',
# max_depth=18,
# min_samples_leaf=2, 
# min_samples_split=3, 
# n_estimators=600
# )

# rfr_model.fit(X_train, y_train)

### linear regressor

In [10]:
# from sklearn.linear_model import LinearRegression

# linear_model = LinearRegression()

# linear_model.fit(X_train,y_train)

### nnls

In [11]:
a = 1.21672907e-01 
b = 0.00000000e+00 
c = 0.00000000e+00 
d = 0.00000000e+00
e = 0.00000000e+00 
f = 0.00000000e+00 
g = 5.16188458e-04 
h = 0.00000000e+00
i = 0.00000000e+00 
j = 3.08837530e-05 
k = 0.00000000e+00 
l = 0.00000000e+00
m = 6.27070417e-02 
n = 6.55179440e-07 
o = 1.39430967e-10

### test

In [13]:
xgbregressor_y_pred = xgbregressor_model.predict(X_test)
dnn_y_pred = dnn_model.predict(X_test_scaled).reshape(-1,)
#linear_y_pred = linear_model.predict(X_test)
nnls_y_pred = a*X_test['lr']+b*X_test['lc']+c*X_test['rc']+d*X_test['ld']+e*X_test['rd']+f*X_test['lnnz']+g*X_test['rnnz']+h*X_test['lr*lc']+i*X_test['lc*rc']+j*X_test['lr*rc']+k*X_test['lr*lc*rc']+l*X_test['ld*rd']+m*X_test['lr*rc*ld*rd']+n*X_test['lr*lc*rc*ld*rd']+o*X_test['lnnz*rnnz']
#rfr_y_pred = rfr_model.predict(X_test)

In [14]:
print(mape_error(y_test,xgbregressor_y_pred))
print(mape_error(y_test,dnn_y_pred))
#print(mape_error(y_test,linear_y_pred))
print(mape_error(y_test,nnls_y_pred))
#print(mape_error(y_test,rfr_y_pred))

9.52215099157472
22.2421426344359
168.302296652129


In [15]:
result_list = {}
mape_list = np.array([])

# 예측값, 실제값을 확인하며 mape 계산 후 mape_list에 삽입 
for idx,value in enumerate(y_test):
    mape_temp = {}
    median_temp = np.array([])
    
    # 각 모델의 예측값
    xgbregressor_predicate = int(xgbregressor_y_pred[idx])
    dnn_predicate = int(dnn_y_pred[idx])
    #linear_predicate = int(linear_y_pred[idx])
    nnls_predicate = int(nnls_y_pred[idx])
    #rfr_predicate = int(rfr_y_pred[idx])
    
    # 각 모델의 예측값을 배열에 삽입
    median_temp = np.append(median_temp, np.array([xgbregressor_predicate,dnn_predicate,nnls_predicate]))
    
    # median 값 사용
    mape = abs((value - np.median(median_temp)) / value) * 100
    
    # mean 값 사용
    # mape = abs((value - np.mean(median_temp)) / value) * 100
    
    mape_temp['xgb_pred'] = xgbregressor_predicate
    mape_temp['dnn_pred'] = dnn_predicate
    #mape_temp['linear_pred'] = linear_predicate
    mape_temp['nnls_pred'] = nnls_predicate
    #mape_temp['rfr_pred'] = rfr_predicate
    mape_temp['--best_pred--'] = np.median(median_temp)
    mape_temp['real'] = value
    mape_temp['mape'] = mape

    mape_list = np.append(mape_list,np.array([mape]))
    result_list[idx] = mape_temp

result_list_sort = sorted(result_list.values(), key=lambda x:(x['mape']), reverse=True)
result_list_sort  

[{'xgb_pred': 3084,
  'dnn_pred': 12788,
  'nnls_pred': 87976,
  '--best_pred--': 12788.0,
  'real': 3095,
  'mape': 313.18255250403877},
 {'xgb_pred': 3683,
  'dnn_pred': 12810,
  'nnls_pred': 55077,
  '--best_pred--': 12810.0,
  'real': 3236,
  'mape': 295.8590852904821},
 {'xgb_pred': 3872,
  'dnn_pred': 12142,
  'nnls_pred': 51722,
  '--best_pred--': 12142.0,
  'real': 3922,
  'mape': 209.58694543600203},
 {'xgb_pred': 4368,
  'dnn_pred': 11726,
  'nnls_pred': 31105,
  '--best_pred--': 11726.0,
  'real': 4131,
  'mape': 183.85378842895184},
 {'xgb_pred': 18357,
  'dnn_pred': 51569,
  'nnls_pred': 72893,
  '--best_pred--': 51569.0,
  'real': 19749,
  'mape': 161.1220821307408},
 {'xgb_pred': 7979,
  'dnn_pred': 17732,
  'nnls_pred': 58413,
  '--best_pred--': 17732.0,
  'real': 7171,
  'mape': 147.27374145865292},
 {'xgb_pred': 5567,
  'dnn_pred': 8458,
  'nnls_pred': 50410,
  '--best_pred--': 8458.0,
  'real': 3651,
  'mape': 131.66255820323198},
 {'xgb_pred': 3916,
  'dnn_pred': 79

In [16]:
# 최적의 pred을 골라냈을 때 mape 평균
print(np.mean(mape_list))

20.655585239033034
